In [ ]:
!pip install datasets trl peft transformers accelerate bitsandbytes

In [ ]:
import torch
import datasets
import transformers
import torch.nn as nn
import pandas as pd
from trl import SFTTrainer
from peft import LoraConfig
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments

In [ ]:
#checking GPU availability
device = 'cpu'
if(torch.cuda.is_available()):
    device = 'cuda'

print(device)

cuda


In [ ]:
#Loading train test datasets
Train_Path = "/content/DFAR_trainset.csv"
Test_Path = "/content/DFAR_trainset.csv"

train_set = pd.read_csv(Train_Path, encoding='utf-8')
test_set = pd.read_csv(Test_Path, encoding='utf-8')

In [ ]:
#Fromating label colume, replacing number with text
train_set['Label'] = train_set['Label'].replace({0: "ethical", 1: "unethical"})
test_set['Label'] = test_set['Label'].replace({0: "ethical", 1: "unethical"})

In [ ]:
#dataset parsing
train_set = datasets.Dataset.from_dict(train_set)
test_set = datasets.Dataset.from_dict(test_set)

In [ ]:
#Loading LLAMA 2 7B model from huggingface in 4 bit quantization
base_model_name = "NousResearch/Llama-2-7b-chat-hf"

llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
#Formating default prompt
def finetune_label_prompts(example):
    output_texts = []
    for i in range(len(example['Statement'])):
        text = f"### Question: Evaluate whether the statement is ethical or unethical without any reason. Statement: {example['Statement'][i]} The statement is \n ### Answer: {example['Label'][i]}"
        output_texts.append(text)
    return output_texts

In [ ]:
# LoRA Config for quantized fine-tuning
peft_parameters = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = "none",
    task_type = "CAUSAL_LM"
)

In [ ]:
# Training Parameters
train_params = TrainingArguments(
    output_dir = "./checkpoints",
    num_train_epochs = 2,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 1,
    optim="paged_adamw_32bit",
    save_steps = 25,
    logging_steps = 25,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = False,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = "constant",
)

In [ ]:
# Trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_set,
    eval_dataset = test_set,
    peft_config=peft_parameters,
    tokenizer=llama_tokenizer,
    args=train_params,
    formatting_func=finetune_label_prompts,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

In [ ]:
#training model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
25,2.016700
50,0.824800
75,1.027600
100,0.718000
125,0.999600
150,0.719000
175,0.909400
200,0.690600
225,0.979400
250,0.663800


TrainOutput(global_step=2250, training_loss=0.7549025026957193, metrics={'train_runtime': 6659.0034, 'train_samples_per_second': 1.352, 'train_steps_per_second': 0.338, 'total_flos': 1.9255204199399424e+16, 'train_loss': 0.7549025026957193, 'epoch': 2.0})

In [ ]:
model.save_pretrained("tuned_Model")